## Validation of EIS data

#### The Kramers-Kronig Relations

Electrochemical impedance spectroscopy (EIS) is built on linear systems theory which requires that the system satisfy conditions of causality, linearity, and stability. The Kramers-Kronig relations consist of a set of transformations that can be used to predict one component of the impedance from the other over the frequency limits from zero to infinity. For example, one might calculate the imaginary component of the impedance from the measured real component, 

$$
Z^{\prime\prime}(\omega) = - \frac{2\omega}{\pi} \int_0^\infty \frac{Z^{\prime}(x) - Z^{\prime}(\omega)}{x^2 - \omega^2}dx
$$

where $Z^{\prime}(\omega)$ and $Z^{\prime\prime}(\omega)$ are the real and imaginary components of the impedance as a function of frequency, $\omega$. Similarly, the real part of the impedance spectrum can be calculated from the imaginary part by

$$
Z^{\prime}(\omega) = Z^{\prime}(\infty) + \frac{2}{\pi} \int_0^\infty{\frac{xZ^{\prime\prime}(x) - \omega Z^{\prime\prime}(\omega)}{x^2 - \omega^2}dx}
$$

The residual error between the predicted and measured impedance can then be used to determine consistency with the Kramers-Kronig relations.

Practically, however, the 0 to $\infty$ frequency range required for integration can be difficult to measure experimentally, so several other methods have been developed to ensure Kramers-Kronig relations are met:

- [Measurement models](#Measurement-models)

- [The Lin-KK method](#The-Lin-KK-method)

#### Measurement models

In [ ]:
import sys

import matplotlib.pyplot as plt
import numpy as np

sys.path.append("../../../")

from impedance import preprocessing
from impedance.models.circuits import CustomCircuit

In [ ]:
# Load data from the example EIS result
f, Z = preprocessing.readCSV("../../../data/exampleData.csv")

# keep only the impedance data in the first quandrant
f, Z = preprocessing.ignoreBelowX(f, Z)

mask = f < 1000
f = f[mask]
Z = Z[mask]

In [ ]:
N = 10

circuit = "R_0"
initial_guess = [0.015]
for i in range(N):
    circuit += f"-p(R_{i % 9 + 1},C_{i % 9 + 1})"
    initial_guess.append(0.03 / N)
    initial_guess.append(10 ** (3 - 6 * i / N))

meas_model = CustomCircuit(initial_guess=initial_guess, circuit=circuit)

In [ ]:
meas_model.fit(f, Z)

print(meas_model)

In [ ]:
from impedance.visualization import plot_nyquist, plot_residuals

res_meas_real = (Z - meas_model.predict(f)).real / np.abs(Z)
res_meas_imag = (Z - meas_model.predict(f)).imag / np.abs(Z)

fig = plt.figure(figsize=(5, 8))
gs = fig.add_gridspec(3, 1)
ax1 = fig.add_subplot(gs[:2, :])
ax2 = fig.add_subplot(gs[2, :])

# plot original data
plot_nyquist(ax1, Z, fmt="s")

# plot measurement model
plot_nyquist(ax1, meas_model.predict(f), fmt="-", scale=1e3, units="\Omega")

ax1.legend(["Data", "Measurement model"], loc=2, fontsize=12)

# Plot residuals
plot_residuals(ax2, f, res_meas_real, res_meas_imag, y_limits=(-2, 2))

plt.tight_layout()
plt.show()

#### The Lin-KK method

The lin-KK method from Schönleber et al. [1] is a quick test for checking the validity of EIS data. The validity of an impedance spectrum is analyzed by its reproducibility by a Kramers-Kronig (KK) compliant equivalent circuit. In particular, the model used in the lin-KK test is an ohmic resistor, $R_{Ohm}$, and $M$ RC elements.

$$
\hat Z = R_{Ohm} + \sum_{k=1}^{M} \frac{R_k}{1 + j \omega \tau_k}
$$

The $M$ time constants, $\tau_k$, are distributed logarithmically,

$$
\tau_1 = \frac{1}{\omega_{max}} ; \tau_M = \frac{1}{\omega_{min}}
; \tau_k = 10^{\log{(\tau_{min}) + \frac{k-1}{M-1}\log{{(
    \frac{\tau_{max}}{\tau_{min}}}})}}
$$

and are not fit during the test (only $R_{Ohm}$ and $R_{k}$ are free parameters).

In order to prevent under- or over-fitting, Schönleber et al. propose using the ratio of positive resistor mass to negative resistor mass as a metric for finding the optimal number of RC elements.

$$
\mu = 1 - \frac{\sum_{R_k \ge 0} |R_k|}{\sum_{R_k < 0} |R_k|}
$$

The argument `c` defines the cutoff value for $\mu$. The algorithm starts at `M = 3` and iterates up to `max_M` until a $\mu < c$ is reached. The default of 0.85 is simply a heuristic value based off of the experience of Schönleber et al.

If the argument `c` is `None`, then the automatic determination of RC elements is turned off and the solution is calculated for `max_M` RC elements. This manual mode should be used with caution as under- and over-fitting should be avoided.


[1] Schönleber, M. et al. A Method for Improving the Robustness of linear Kramers-Kronig Validity Tests. Electrochimica Acta 131, 20–27 (2014) [doi: 10.1016/j.electacta.2014.01.034](https://doi.org/10.1016/j.electacta.2014.01.034).


In [ ]:
import sys

import matplotlib.pyplot as plt
import numpy as np

sys.path.append("../../../")

from impedance.validation import linKK

In [ ]:
# Load data from the example EIS result
f, Z = preprocessing.readCSV("../../../data/exampleData.csv")

# keep only the impedance data in the first quandrant
f, Z = preprocessing.ignoreBelowX(f, Z)

mask = f < 1000
f = f[mask]
Z = Z[mask]

In [ ]:
M, mu, Z_linKK, res_real, res_imag = linKK(
    f, Z, c=0.5, max_M=100, fit_type="complex", add_cap=True
)

print("\nCompleted Lin-KK Fit\nM = {:d}\nmu = {:.2f}".format(M, mu))

In [ ]:
from impedance.visualization import plot_nyquist, plot_residuals

fig = plt.figure(figsize=(5, 8))
gs = fig.add_gridspec(3, 1)
ax1 = fig.add_subplot(gs[:2, :])
ax2 = fig.add_subplot(gs[2, :])

# plot original data
plot_nyquist(ax1, Z, fmt="s")

# plot measurement model
plot_nyquist(ax1, Z_linKK, fmt="-", scale=1e3, units="\Omega")

ax1.legend(["Data", "Lin-KK model"], loc=2, fontsize=12)

# Plot residuals
plot_residuals(ax2, f, res_real, res_imag, y_limits=(-2, 2))

plt.tight_layout()
plt.show()